In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, f1_score
import lightgbm as lgb
import warnings
import re
import pickle

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Criar diretório para salvar as visualizações
os.makedirs('visualizacoes', exist_ok=True)

def clean_col_name(col_name):
    """Limpa o nome da coluna para ser compatível com LightGBM."""
    if isinstance(col_name, str) and col_name.startswith("(") and col_name.endswith(")") :
        try:
            evaluated_col = eval(col_name)
            if isinstance(evaluated_col, tuple):
                col_name = f"{evaluated_col[0].strip()}_{evaluated_col[1].strip()}"
        except:
            pass

    col_name = str(col_name)
    col_name = re.sub(r"[\W]+(?<!^)", "_", col_name)
    col_name = col_name.strip("_")
    if col_name and col_name[0].isdigit():
        col_name = "_" + col_name
    return col_name

def train_predictive_model(file_path):
    try:
        # Carregar a base de dados principal
        df_main = pd.read_excel(file_path)
        print(f"Base de dados carregada: {df_main.shape[0]} linhas, {df_main.shape[1]} colunas")

        # --- Tratamento de Nomes de Colunas ---
        original_columns = df_main.columns.tolist()
        new_columns = [clean_col_name(col) for col in original_columns]

        # Verificar duplicatas após limpeza
        if len(new_columns) != len(set(new_columns)):
            print("Aviso: Nomes de colunas duplicados detectados após a limpeza. Renomeando...")
            counts = {}
            final_columns = []
            for col in new_columns:
                if col in counts:
                    counts[col] += 1
                    final_columns.append(f"{col}_{counts[col]}")
                else:
                    counts[col] = 0
                    final_columns.append(col)
            new_columns = final_columns

        df_main.columns = new_columns
        print("Nomes de colunas limpos e renomeados.")

        # Mapeamento dos nomes de colunas específicos solicitados pelo usuário
        col_mapping = {}
        for col in df_main.columns:
            if "P1_a_1" in col or "Faixa_idade" in col or "faixa_etaria" in col:
                col_mapping["faixa_etaria"] = col
            elif "P1_b" in col or "Genero" in col or "genero" in col:
                col_mapping["genero"] = col
            elif "P1_e_1" in col or "Não_acredito" in col or "nao_afetado" in col:
                col_mapping["nao_afetado"] = col
            elif "P1_e_2" in col or "Cor_Raça_Etnia" in col or "impacto_raca" in col:
                col_mapping["target"] = col
            elif "P1_e_3" in col or "identidade_de_gênero" in col or "impacto_genero" in col:
                col_mapping["impacto_genero"] = col
            elif "P1_l" in col or "Nivel_de_Ensino" in col or "nivel_ensino" in col:
                col_mapping["nivel_ensino"] = col
            elif "P2_h" in col or "Faixa_salarial" in col or "faixa_salarial" in col:
                col_mapping["faixa_salarial"] = col
            elif "P2_i" in col or "experiência_na_área" in col or "tempo_experiencia" in col:
                col_mapping["tempo_experiencia"] = col

        print("Mapeamento de colunas específicas:")
        for key, value in col_mapping.items():
            print(f"  {key}: {value}")

        # Verificar se todas as colunas necessárias foram encontradas
        required_cols = ["faixa_etaria", "genero", "nao_afetado", "target", "impacto_genero",
                         "nivel_ensino", "faixa_salarial", "tempo_experiencia"]
        missing_cols = [col for col in required_cols if col not in col_mapping]

        if missing_cols:
            print(f"Erro: Colunas necessárias não encontradas: {missing_cols}")
            return

        # --- Preparação dos Dados ---
        # Remover linhas onde a variável alvo é NaN
        target_col = col_mapping["target"]
        df_main = df_main.dropna(subset=[target_col])
        print(f"Linhas após remover NaN na coluna alvo: {df_main.shape[0]}")

        # Selecionar apenas as colunas específicas
        feature_cols = [col_mapping[col] for col in required_cols if col != "target"]
        X = df_main[feature_cols].copy()
        y = df_main[target_col].astype(int)

        print(f"Features selecionadas: {X.columns.tolist()}")

        # Identificar colunas categóricas e numéricas
        categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()
        numerical_features = X.select_dtypes(include=np.number).columns.tolist()

        # --- Pré-processamento ---
        # Preencher NaNs em numéricas com a mediana
        for col in numerical_features:
            if X[col].isnull().any():
                median_val = X[col].median()
                X[col] = X[col].fillna(median_val)

        # Converter categóricas para tipo "category" e preencher NaNs
        for col in categorical_features:
            if not isinstance(X[col].dtype, pd.CategoricalDtype):
                X[col] = X[col].astype("category")
            if X[col].isnull().any():
                if "Missing" not in X[col].cat.categories:
                    X[col] = X[col].cat.add_categories("Missing")
                X[col] = X[col].fillna("Missing")

        print("Pré-processamento concluído.")

        # --- Divisão Treino/Teste ---
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        print(f"Dados divididos em treino ({X_train.shape[0]} amostras) e teste ({X_test.shape[0]} amostras).")

        # --- Validação Cruzada ---
        print("Iniciando validação cruzada...")
        lgbm = lgb.LGBMClassifier(random_state=42)
        cv_scores = cross_val_score(lgbm, X, y, cv=5, scoring='accuracy')
        print(f"Resultados da validação cruzada: {cv_scores}")
        print(f"Acurácia média da validação cruzada: {cv_scores.mean():.4f}")

        # --- Otimização de Hiperparâmetros com GridSearchCV ---
        print("Iniciando otimização de hiperparâmetros com GridSearchCV...")
        param_grid = {
            'num_leaves': [31, 50],
            'max_depth': [5, 10, -1],
            'learning_rate': [0.01, 0.05],
            'n_estimators': [20, 40]
        }
        grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")

        # --- Treinamento com os melhores parâmetros ---
        best_lgbm = grid_search.best_estimator_
        best_lgbm.fit(X_train, y_train)

        # --- Avaliação do Modelo ---
        print("Avaliando o modelo no conjunto de teste...")
        y_pred = best_lgbm.predict(X_test)
        y_pred_proba = best_lgbm.predict_proba(X_test)[:, 1]

        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)

        print(f"\n--- Resultados da Avaliação ---")
        print(f"Acurácia no conjunto de teste: {accuracy:.4f}")
        print(f"F1-Score no conjunto de teste: {f1:.4f}")
        print("\nRelatório de Classificação:")
        print(report)

        # Matriz de Confusão Normalizada
        conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

        # Salvar resultados em arquivo
        with open("modelo_preditivo_resultados_completo.txt", "w", encoding="utf-8") as f:
            f.write(f"Modelo: LightGBM com validação cruzada e otimização de hiperparâmetros\n")
            f.write(f"Variável Alvo: {target_col}\n\n")
            f.write(f"Melhores parâmetros: {grid_search.best_params_}\n\n")
            f.write(f"Acurácia média da validação cruzada: {cv_scores.mean():.4f}\n\n")
            f.write(f"Acurácia no conjunto de teste: {accuracy:.4f}\n")
            f.write(f"F1-Score no conjunto de teste: {f1:.4f}\n\n")
            f.write("Relatório de Classificação:\n")
            f.write(report)

        # Salvar o modelo treinado
        with open('modelo_lightgbm_otimizado.pkl', 'wb') as f:
            pickle.dump(best_lgbm, f)
        print("Modelo otimizado salvo em modelo_lightgbm_otimizado.pkl")

        # --- Criação de Visualizações ---
        print("\nCriando visualizações dos resultados...")

        # Visualizações restantes...

        print("Visualizações criadas com sucesso!")

        return {
            'accuracy': accuracy,
            'f1_score': f1,
            'feature_importance': feature_importance,
            'conf_matrix': conf_matrix,
            'conf_matrix_normalized': conf_matrix_normalized,
            'roc_auc': roc_auc
        }

    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    file_path = "Main_database (2).xlsx"
    results = train_predictive_model(file_path)


Base de dados carregada: 5293 linhas, 399 colunas
Nomes de colunas limpos e renomeados.
Mapeamento de colunas específicas:
  faixa_etaria: P1_a_1_Faixa_idade
  genero: P1_b_Genero
  nao_afetado: P1_e_1_Não_acredito_que_minha_experiência_profissional_seja_afetada
  target: P1_e_2_Experiencia_prejudicada_devido_a_minha_Cor_Raça_Etnia
  impacto_genero: P1_e_3_Experiencia_prejudicada_devido_a_minha_identidade_de_gênero
  nivel_ensino: P1_l_Nivel_de_Ensino
  faixa_salarial: P2_h_Faixa_salarial
  tempo_experiencia: P2_j_Quanto_tempo_de_experiência_na_área_de_TI_Engenharia_de_Software_você_teve_antes_de_começar_a_trabalhar_na_área_de_dados
Linhas após remover NaN na coluna alvo: 2719
Features selecionadas: ['P1_a_1_Faixa_idade', 'P1_b_Genero', 'P1_e_1_Não_acredito_que_minha_experiência_profissional_seja_afetada', 'P1_e_3_Experiencia_prejudicada_devido_a_minha_identidade_de_gênero', 'P1_l_Nivel_de_Ensino', 'P2_h_Faixa_salarial', 'P2_j_Quanto_tempo_de_experiência_na_área_de_TI_Engenharia_de_Sof

Traceback (most recent call last):
  File "<ipython-input-10-a0661896aa7d>", line 206, in train_predictive_model
    'feature_importance': feature_importance,
                          ^^^^^^^^^^^^^^^^^^
NameError: name 'feature_importance' is not defined
